In [ ]:
import dill
import os
from pathlib import Path
import itertools

#### Notes on file structure

- results are stored in ../results/results
- figures are stored in ../results/figures
- folder structure is based on data (i.e. start/end point, number of datapoints and noise level)
- file names are based on the actual model i.e. system name, l1 and l2 values 
- file names are separated by "_"
- relevant results files are ...
    - f"{filename_addendum}_MLL.pkl"
    - f"{filename_addendum}_MLL_logs.pkl"
    - f"{filename_addendum}_MAP.pkl"
    - f"{filename_addendum}_MAP_logs.pkl"
    - f"{filename_addendum}_mean_ode_satisfaction_error_MLL.pkl"
    - f"{filename_addendum}_mean_ode_satisfaction_error_MAP.pkl"
    - f"{filename_addendum}_sample_ode_satisfaction_error_MLL.pkl"
    - f"{filename_addendum}_sample_ode_satisfaction_error_MAP.pkl"
    - f"{filename_addendum}_MLL_model_train_MSEs.pkl"
    - f"{filename_addendum}_MAP_model_train_MSEs.pkl"
    - f"{filename_addendum}_MLL_model_test_MSEs.pkl"
    - f"{filename_addendum}_MAP_model_test_MSEs.pkl"
- relevant figure files are ...
    - f"MLL_model_posterior_{system_name}_l1-{l1_param_val}_l2-{l2_param_val}_{START}-{END}-{COUNT}_{noise_level}.png"
    - f"MAP_model_posterior_{system_name}_l1-{l1_param_val}_l2-{l2_param_val}_{START}-{END}-{COUNT}_{noise_level}.png"

#### Notes on relevant metrics / summaries / statistics
- Visualize the trends for all metrics over increasing training data and noise
    - Maybe plot number of datapoints on X and noise level on Y and color the points for MSE/MLL/MAP/... ? (This only works if working with a single model, as we have overlap otherwise)
- Display some instances of good/bad trainings for both a lot and few datapoints
- Show the difference that we get when we have few datapoints at the start and across the whole domain
- Lineplot with different colors for different models, X is number of datapoints, Y is MSE/MLL/MAP/...
    - When comparing the ODE satisfaction use a log-scale
- Look at some interesting individual cases of the data
    - Highlight the slow speed of the moon system in changing its values
    - 

In [ ]:
# Helper function to load a specific result from a model
def load_model_result(model_name, result_name, results_path=None):
    if results_path is None:
        results_path = Path.cwd()
        results_path = results_path.joinpath('results').joinpath("results")
    # Construct the file path based on the model and result names
    file_path = results_path.joinpath(f'{model_name}_{result_name}.pkl')
    
    # Check if the file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Result file '{file_path}' does not exist.")
    
    # Load the result using dill
    with open(file_path, 'rb') as file:
        result = dill.load(file)
    
    return result

load_model_result("Bipendulum_l1-1.0_l2-2.0", "MLL", results_path=Path.cwd().parent.joinpath("results").joinpath("results").joinpath("2-3-1_0.0"))

In [ ]:
def construct_model_name(model_name, l1, l2):
    """
    Constructs a model name based on the provided parameters.
    
    Args:
        model_name (str): The base name of the model.
        l1 (float): The first parameter for the model.
        l2 (float): The second parameter for the model.
    
    Returns:
        str: The constructed model name.
    """
    return f"{model_name}_l1-{l1}_l2-{l2}"


def construct_experiment_name(start, end, count, noise):
    return f"{start}-{end}-{count}_{noise}"


def construct_experiment_path(start, end, count, noise, results_path=None):
    """
    Constructs the path for the experiment results based on the provided parameters.
    
    Args:
        start (float): The start value for the experiment.
        end (float): The end value for the experiment.
        count (int): The number of samples in the experiment.
        noise (float): The noise level in the experiment.
        result_path (str, optional): The base path for results. Defaults to None.
    
    Returns:
        str: The constructed path for the experiment results.
    """
    if results_path is None:
        results_path = Path.cwd()
        results_path = results_path.joinpath('results').joinpath("results")
    return results_path.joinpath(f"{start}-{end}-{count}_{noise}")


print(construct_experiment_path(2, 3, 100, 0.1, results_path=Path.cwd().parent.joinpath("results").joinpath("results")).joinpath(construct_model_name("Bipendulum", 1.0, 2.0)))
print(construct_experiment_path(2, 3, 100, 0.1, results_path=Path.cwd().parent.joinpath("results").joinpath("results")).exists())

In [ ]:
# Easy Metric Call Dict
emcd = {"MLL" : "MLL",
"MLL logs": "MLL_logs",
"MAP" : "MAP",
"MAP logs": "MAP_logs",
"mean MLL ODE error": "mean_ode_satisfaction_error_MLL",
"mean MAP ODE error": "mean_ode_satisfaction_error_MAP",
"sample MLL ODE error": "sample_ode_satisfaction_error_MLL",
"sample MAP ODE error": "sample_ode_satisfaction_error_MAP",
"MLL train MSEs": "MLL_model_train_MSEs",
"MAP train MSEs": "MAP_model_train_MSEs",
"MLL test MSEs": "MLL_model_test_MSEs",
"MAP test MSEs": "MAP_model_test_MSEs"}

# More used as a reference than for actual use
all_model_names = ["Bipendulum", "Bipendulum first equation", "Bipendulum second equation", "Bipendulum Sum eq2 diffed", "Bipendulum moon gravitation", "Bipendulum Parameterized", "No system"]

all_l1_l2_combinations = list([[1.0, 2.0], [1.0, 3.0], [2.0, 3.0]])

all_model_settings = list(itertools.chain(itertools.product(["Bipendulum", "Bipendulum first equation", "Bipendulum second equation", "Bipendulum Sum eq2 diffed", "Bipendulum moon gravitation"], [[1.0, 2.0], [1.0, 3.0], [2.0, 3.0]]), [("Bipendulum Parameterized", [1.0, 2.0]),  ("No system", [1.0, 2.0])]))

all_ranges = [(2, 12), (2, 3)]
all_dataset_sizes = [1, 2, 5, 10, 20, 50, 100]
all_noises = [0.0, 0.1, 0.2, 0.3]

all_experiment_settings = list(itertools.product([(2, 12), (2, 3)], [1, 2, 5, 10, 20, 50, 100], [0.0, 0.1, 0.2, 0.3]))

In [ ]:

def unpack_experiment_setting(experiment_setting):
    return *experiment_setting[0], *experiment_setting[1:]

def unpack_model_setting(model_setting):
    return model_setting[0], *model_setting[1]


In [ ]:
base_results_path = Path.cwd().parent.joinpath("results").joinpath("results")
construct_experiment_path()

In [ ]:
load_model_result(f"{construct_model_name(all_model_settings[0][0], *all_model_settings[0][1])}", emcd['MLL'],  construct_experiment_path(*all_experiment_settings[0][0], *all_experiment_settings[0][1:], results_path=base_results_path))


In [ ]:
load_model_result(f"{construct_model_name(*unpack_model_setting(all_model_settings[0]))}", emcd['MLL'],  construct_experiment_path(*unpack_experiment_setting(all_experiment_settings[0]), results_path=base_results_path))

In [ ]:
for experiment_setting, model_setting in itertools.product(all_experiment_settings, all_model_settings):
    try:
        model_name = construct_model_name(*unpack_model_setting(model_setting))
        experiment_path = construct_experiment_path(*unpack_experiment_setting(experiment_setting), results_path=base_results_path)
        result = load_model_result(f"{model_name}", emcd['MLL'], experiment_path)
        print(f"Successfully loaded result for {model_setting} in experiment {experiment_setting}")
    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"An error occurred while loading result for {model_setting} in experiment {experiment_setting}: {e}")